# Figures

Notebook to start making figures for Tmax paper.

Cascade Tuholske 2019-09-08

#### Depdendencies

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Load and arrange data

In [2]:
# file paths

TDATA_IN = '/home/cascade/projects/data_out/'
GHS_DATA_IN = '/home/CHIRTS/Tmax/cascade/GHS-UCDB/'

In [3]:
# file names

chirts_daily_fn = '20190831_TMax-GHS_TotEvents83-2016.csv'
GHS_fn = 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp'

In [4]:
# Open daily and GHS_UCDB

df_chirts = pd.read_csv(TDATA_IN+chirts_daily_fn)
print(df_chirts.head())

df_ghs = gpd.read_file(GHS_DATA_IN+GHS_fn)
print(df_ghs.head())

   Unnamed: 0  ID_HDC_G0 CTR_MN_NM  1983  1984  1985  1986  1987  1988  1989  \
0           0       4362    Russia   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1           1       4884    Russia   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2           2       4473    Russia   NaN   NaN   NaN   NaN   NaN   1.0   NaN   
3           3       4502    Russia   NaN   NaN   NaN   NaN   NaN   1.0   NaN   
4           4       4641    Russia   NaN   NaN   NaN   NaN   NaN   1.0   NaN   

   ...  2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  
0  ...   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
1  ...   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
2  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3  ...   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
4  ...   NaN   NaN   NaN   3.0   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 37 columns]
   ID_HDC_G0  QA2_1V  AREA  BBX_LATMN  BBX_LONMN  BBX_LATMX  BBX_LONMX  \
0       

In [5]:
# Map in city populations

# Get Pop
df_pop = pd.DataFrame()
df_pop['ID_HDC_G0'] = df_ghs['ID_HDC_G0']
df_pop['P75'] = df_ghs['P75']
df_pop['P90'] = df_ghs['P90']
df_pop['P00'] = df_ghs['P00']
df_pop['P15'] = df_ghs['P15']

df_pop.head()

,ID_HDC_G0,P75,P90,P00,P15
0,5782,76680.403870,77349.278442,75261.170776,69023.903442
1,3316,432518.760010,399369.630417,350416.157928,281927.639362
2,5645,83301.193208,78605.444337,73295.000717,96123.248169
3,3185,59961.838394,67281.163544,73833.521271,83504.834900
4,3539,216241.069946,203083.534027,181690.372681,155063.227570


In [6]:
# merge

df_chirts_merge = pd.merge(df_pop, df_chirts, on='ID_HDC_G0', how = 'inner')


In [7]:
df_chirts_merge.head()

,ID_HDC_G0,P75,P90,P00,P15,Unnamed: 0,CTR_MN_NM,1983,1984,1985,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,4362,208910.092712,188073.401855,161281.682434,128157.266022,0,Russia,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,4884,112132.820587,108378.446289,98760.031052,84903.883148,1,Russia,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,4473,480713.235840,438750.923782,377515.400574,298582.390823,2,Russia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4502,86729.534424,80556.967285,70608.384003,57013.894836,3,Russia,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4641,87747.461609,93024.031006,90008.577820,84516.540771,4,Russia,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Make people days

df_Tpop = pd.DataFrame()
df_Tpop['ID_HDC_G0'] = df_chirts_merge['ID_HDC_G0']
df_Tpop['CTR_MN_NM'] = df_chirts_merge['CTR_MN_NM']
df_Tpop['P75'] = df_chirts_merge['P75']
df_Tpop['P90'] = df_chirts_merge['P90']
df_Tpop['P00'] = df_chirts_merge['P00']
df_Tpop['P15'] = df_chirts_merge['P15']
df_Tpop['1983'] = df_chirts_merge['1983']
df_Tpop['1990'] = df_chirts_merge['1990']
df_Tpop['2000'] = df_chirts_merge['2000']
df_Tpop['2015'] = df_chirts_merge['2015']

In [9]:
df_Tpop['PD_1983'] = df_Tpop['P75'] * df_Tpop['1983']
df_Tpop['PD_1990'] = df_Tpop['P90'] * df_Tpop['1990']
df_Tpop['PD_2000'] = df_Tpop['P00'] * df_Tpop['2000']
df_Tpop['PD_2015'] = df_Tpop['P15'] * df_Tpop['2015']


In [10]:
df_Tpop[2500:2600]

,ID_HDC_G0,CTR_MN_NM,P75,P90,P00,P15,1983,1990,2000,2015,PD_1983,PD_1990,PD_2000,PD_2015
2500,11428,China,151313.624191,180821.804626,184990.786621,170883.531250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2501,11531,China,311383.498984,375082.377702,392898.051056,378537.643524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2502,6848,India,114265.215576,181382.671143,218643.872314,235628.027588,1.0,7.0,9.0,28.0,1.142652e+05,1.269679e+06,1.967795e+06,6.597585e+06
2503,10094,India,19613.737503,41933.777344,50638.237000,71062.891724,1.0,1.0,1.0,NaN,1.961374e+04,4.193378e+04,5.063824e+04,NaN
2504,3978,Egypt,24540.694153,42675.919800,52328.870117,69414.006836,11.0,5.0,4.0,30.0,2.699476e+05,2.133796e+05,2.093155e+05,2.082420e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,8394,India,26098.964844,46437.875488,55266.675293,66158.498047,3.0,3.0,1.0,12.0,7.829689e+04,1.393136e+05,5.526668e+04,7.939020e+05
2596,12593,China,119824.209682,187267.417145,233803.872742,300490.412781,1.0,NaN,1.0,NaN,1.198242e+05,NaN,2.338039e+05,NaN
2597,7318,India,154524.190277,187437.569824,181578.716858,84017.209229,23.0,32.0,28.0,43.0,3.554056e+06,5.998002e+06,5.084204e+06,3.612740e+06
2598,3966,Egypt,188502.884985,290004.112202,351349.791428,474894.187260,13.0,9.0,7.0,34.0,2.450538e+06,2.610037e+06,2.459449e+06,1.614640e+07


In [ ]:
# Write it out

# df_Tpop.to_csv(TDATA_IN+'CHIRTS-GHS-PeopleDays.csv')

#### Sum People Days and Plot it <<<<--- CASCADE START HERE AND CHECK SUMS

In [19]:
df_ghs.head()

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE901,SDG_A2G14,SDG_OS15MX,geometry
0,5782,1,23,69.303014,88.108033,69.362302,88.296220,69.333682,88.205172,Russia,...,0.0,0.0,0.009675,2,available,4.314549923,-1.007979770256988,0.000000,86.61,"POLYGON ((88.1393063460871 69.36230222917577, ..."
1,3316,1,72,68.866046,32.988859,69.054626,33.160407,68.955354,33.078645,Russia,...,0.0,0.0,0.007464,2,available,3.012389898,-0.1770706149298426,0.197219,61.42,"POLYGON ((33.06495302820559 69.05462590164764,..."
2,5645,1,36,66.067184,76.486580,66.101046,76.816776,66.083799,76.646580,Russia,...,0.0,0.0,0,0,available,10.88630009,2.227581243555402,0.184268,32.17,"POLYGON ((76.62270472801578 66.10104608358459,..."
3,3185,1,37,64.980756,25.406867,65.058622,25.561199,65.019378,25.482396,Finland,...,0.0,0.0,0.010561,2,available,3.785350084,0.4888695146841359,0.369999,66.81999999999999,"POLYGON ((25.44769102312469 65.05862213014839,..."
4,3539,1,51,64.537350,39.738776,64.614759,39.933162,64.572142,39.831477,Russia,...,0.0,0.0,0,0,available,9.159580231,-0.4058050539827563,0.560615,58.41,"POLYGON ((39.81130040982809 64.6147588063249, ..."


In [13]:
year = '1983'
pd_year = 'PD_1983'

pd_tot = df_Tpop.groupby(year)[pd_year].sum()
pd_range = (list(range(1,len(pd_tot)+1)))

print(len(pd_tot))
print(len(pd_range))

test = pd.DataFrame()
test['pd_tot'] = pd_tot / 10**6
test['pd_range'] = pd_range

tot = test.pd_tot.sum()
print(tot*10**6/10**9) 

139
139
7.983131124426917


In [ ]:
# make plot

sns.set(font_scale=1)
# fig, ax = plt.subplots(nrows=1, ncols=1)
fig = plt.subplots(figsize=[8,8])

ax = sns.barplot(x='pd_range', y='pd_tot', data = test)

# Colors
ax.set_facecolor('White')

# Labels
plt.xlabel('Number of Days Per Year > 40.6C')
plt.ylabel('Total Number of Urban Residents Exposed (millions)')
plt.title('Number of Urban Residents Exposed to Days >40.6 in '+year+' (3.1 Billion total)')

# ticks
tick = list(range(0,200, 50))
plt.xticks(tick, tick)
